# DataAnswerBot

1. We want this bot can do the sql or other execution then answer the question based on the result.

In [1]:
import os
from utils import read_config, MySQLAgent
import openai
from llama_index.llms.openai import OpenAI
import google.generativeai as genai
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)


from llama_index.core import SQLDatabase, Settings
from llama_index.core.query_engine import NLSQLTableQueryEngine, RouterQueryEngine
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import QueryEngineTool
from llama_index.core.selectors import LLMSingleSelector

from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column
)

e:\miniconda3\envs\bibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
info_path = '.env/info.json'
info = read_config(info_path)

O_KEY = info['okey']
os.environ["OPENAI_API_KEY"] = O_KEY
openai.api_key = os.environ["OPENAI_API_KEY"]

conn_info = info['Crawler_mysql_conn_info_Crawler']
sql_agent = MySQLAgent(info['Crawler_mysql_conn_info_Crawler'])

In [3]:
sql_query = """
SELECT companyname, judgement_no, subjectkey, judgement_hyperlink as link
FROM judgement_result
WHERE judgement_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 5 YEAR);
"""

df_judge = sql_agent.read_table(query=sql_query)
df_judge

,companyname,judgement_no,subjectkey,link
0,新光醫療財團法人,臺灣新北地方法院 108 年度司促字第 32871 號民事裁定,支付命令,https://judgment.judicial.gov.tw/FJUD/data.asp...
1,新光醫療財團法人,臺灣新北地方法院 108 年度訴字第 3634 號民事判決,侵權行為損害賠償,https://judgment.judicial.gov.tw/FJUD/data.asp...
2,新光醫療財團法人,臺灣新北地方法院 112 年度訴字第 1821 號民事判決,侵權行為損害賠償,https://judgment.judicial.gov.tw/FJUD/data.asp...
3,新光醫療財團法人,臺灣新北地方法院 113 年度司促字第 14250 號民事裁定,支付命令,https://judgment.judicial.gov.tw/FJUD/data.asp...
4,新光醫療財團法人,臺灣新北地方法院 113 年度司促字第 15570 號民事裁定,支付命令,https://judgment.judicial.gov.tw/FJUD/data.asp...
...,...,...,...,...
11662,吳秉峰,臺灣雲林地方法院 109 年度重訴字第 8 號民事判決,第三人異議之訴,https://judgment.judicial.gov.tw/FJUD/data.asp...
11663,王士侑,臺灣雲林地方法院 109 年度司促字第 5813 號民事其他文書,支付命令,https://judgment.judicial.gov.tw/FJUD/data.asp...
11664,王仁松,臺灣臺中地方法院 109 年度司促字第 37679 號民事其他文書,支付命令,https://judgment.judicial.gov.tw/FJUD/data.asp...
11665,王仁松,臺灣臺中地方法院 110 年度司促字第 15097 號民事其他文書,支付命令,https://judgment.judicial.gov.tw/FJUD/data.asp...


In [4]:
df_judge.head(30)

,companyname,judgement_no,subjectkey,link
0,新光醫療財團法人,臺灣新北地方法院 108 年度司促字第 32871 號民事裁定,支付命令,https://judgment.judicial.gov.tw/FJUD/data.asp...
1,新光醫療財團法人,臺灣新北地方法院 108 年度訴字第 3634 號民事判決,侵權行為損害賠償,https://judgment.judicial.gov.tw/FJUD/data.asp...
2,新光醫療財團法人,臺灣新北地方法院 112 年度訴字第 1821 號民事判決,侵權行為損害賠償,https://judgment.judicial.gov.tw/FJUD/data.asp...
3,新光醫療財團法人,臺灣新北地方法院 113 年度司促字第 14250 號民事裁定,支付命令,https://judgment.judicial.gov.tw/FJUD/data.asp...
4,新光醫療財團法人,臺灣新北地方法院 113 年度司促字第 15570 號民事裁定,支付命令,https://judgment.judicial.gov.tw/FJUD/data.asp...
5,新光醫療財團法人,板橋簡易庭 113 年度板簡字第 367 號民事判決,侵權行為損害賠償（交通）,https://judgment.judicial.gov.tw/FJUD/data.asp...
6,新光醫療財團法人,臺灣士林地方法院 109 年度訴字第 192 號民事判決,損害賠償,https://judgment.judicial.gov.tw/FJUD/data.asp...
7,新光醫療財團法人,臺灣士林地方法院 109 年度訴字第 445 號民事判決,清償債務,https://judgment.judicial.gov.tw/FJUD/data.asp...
8,新光醫療財團法人,臺灣士林地方法院 109 年度訴字第 990 號民事判決,損害賠償,https://judgment.judicial.gov.tw/FJUD/data.asp...
9,新光醫療財團法人,臺灣士林地方法院 110 年度簡字第 59 號民事判決,給付保險金,https://judgment.judicial.gov.tw/FJUD/data.asp...


In [5]:
sql_query = """
SELECT COUNT(*) AS 判決次數 
FROM judgement_result 
WHERE CompanyName = '新光醫療財團法人' AND SUBJECTKEY LIKE '%%侵權行為損害賠償%%';
"""
sql_agent.read_table(query=sql_query)

,判決次數
0,4


In [6]:
# llm = ChatGoogleGenerativeAI(
#     model="gemini-pro",
#     temperature=0,
#     convert_system_message_to_human=True,
#     safety_settings={
#         HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
#         HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
#         HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
#     },
# )

llm =  OpenAI(temperature=0.1, model="gpt-4o-mini")

In [7]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embed_model = HuggingFaceEmbedding(model_name=model_name)

In [8]:
user = conn_info['user']
pw = conn_info['pw']
host = conn_info['host']
port = conn_info['port']
database = conn_info['database']
connection_string = f"mysql+pymysql://{user}:{pw}@{host}:{port}/{database}?charset=utf8mb4"
engine = create_engine(connection_string)
metadata_obj = MetaData()

tables = ['judgement_result']

sql_database = SQLDatabase(engine, include_tables=tables)

Settings.llm = llm
Settings.embed_model = embed_model
sql_query_engine = NLSQLTableQueryEngine(sql_database=sql_database, tables=tables)

In [9]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=('useful for translating a natural language query into SQL query over.')
)

In [10]:
# prompt_template_str = """
# ####基本描述
# 你現在要根據Question用SQL來查找Database中的資料。
# 目前使用的table叫"judgement_result"，指的是各公司被法院判決的結果；其中companyname代表公司的名稱，subjectkey代表被判決理由。
# 舉例如下:
# 案例1:
# ####Quesiton: 新光醫療財團法人有幾筆被判決資料
# ####Diagnosis: 公司名稱:新光醫療財團法人，判決資料:計算有幾筆Data
# ####SQL: 根據上面Diagnosis的結果SQL如下 "SELECT\n  COUNT(*)\nFROM judgement_result\nWHERE\n  CompanyName = "新光醫療財團法人";"

# 案例2:
# ####Quesiton: 新光醫療財團法人被判侵權行為損害賠償的次數
# ####Diagnosis: 公司名稱:新光醫療財團法人，判決理由:侵權行為損害賠償，次數:計算次數
# ####SQL: 根據上面Diagnosis的結果SQL如下 "SELECT\n  COUNT(*) AS companyname_count\nFROM judgement_result\nWHERE\n  CompanyName = "新光醫療財團法人" AND subjectkey = "侵權行為損害賠償";"

# """

In [11]:
# prompt_template_str = """
# ####基本描述
# 你現在要根據Question用SQL來查找Database中的資料。
# 目前使用的table叫"judgement_result"，指的是各公司被法院判決的結果；其中companyname代表公司的名稱，subjectkey代表被判決理由。
# 舉例如下:
# 案例1:
# ####Quesiton: 新光醫療財團法人有幾筆被判決資料
# ####Diagnosis: 公司名稱:新光醫療財團法人，判決資料:計算有幾筆Data


# 案例2:
# ####Quesiton: 新光醫療財團法人被判侵權行為損害賠償的次數
# ####Diagnosis: 公司名稱:新光醫療財團法人，判決理由(subjectkey):侵權行為損害賠償，次數:計算次數

# """

In [15]:
prompt_template_str = """
###基本描述
你現在要根據Question用SQL來查找Database中的資料。
目前使用的table叫"judgement_result"，指的是各公司被法院判決的結果；其中companyname代表公司的名稱，subjectkey代表被判決理由。

###情況判別
如果Query中沒有出現被判決的理由則不要對subjectkey做篩選
如果Query中出現被判決的理由不存在subjectkey中，則告訴找不到所查找的判決理由

"""

In [12]:
query_engine =  RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[sql_tool]
)

In [13]:
query = """新光醫療財團法人中被判決理由是侵權行為損害賠償的次數"""
response = query_engine.query(query)
response.response

'新光醫療財團法人因侵權行為損害賠償被判決的次數為4次。'

In [14]:
response

Response(response='新光醫療財團法人因侵權行為損害賠償被判決的次數為4次。', source_nodes=[NodeWithScore(node=TextNode(id_='1dd3444f-80c0-4971-8b49-81f1256339b7', embedding=None, metadata={'sql_query': "SELECT COUNT(*) AS 判決次數 FROM judgement_result WHERE CompanyName = '新光醫療財團法人' AND SUBJECTKEY LIKE '%侵權行為損害賠償%';", 'result': [(4,)], 'col_keys': ['判決次數']}, excluded_embed_metadata_keys=['sql_query', 'result', 'col_keys'], excluded_llm_metadata_keys=['sql_query', 'result', 'col_keys'], relationships={}, text='[(4,)]', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'1dd3444f-80c0-4971-8b49-81f1256339b7': {'sql_query': "SELECT COUNT(*) AS 判決次數 FROM judgement_result WHERE CompanyName = '新光醫療財團法人' AND SUBJECTKEY LIKE '%侵權行為損害賠償%';", 'result': [(4,)], 'col_keys': ['判決次數']}, 'sql_query': "SELECT COUNT(*) AS 判決次數 FROM judgement_result WHERE CompanyName = '新光醫療財團法人' AND SUBJECTKEY

In [44]:
metadata = response.metadata
first_key = next(iter(metadata))
metadata[first_key]['sql_query']

"SELECT COUNT(*) AS 判決次數 FROM judgement_result WHERE CompanyName = '新光醫療財團法人' AND SUBJECTKEY LIKE '%侵權行為損害賠償%';"

In [22]:
query = """華盛螺絲有幾筆裁罰紀錄"""
response = query_engine.query(query)

In [23]:
response

Response(response='華盛螺絲目前有24筆裁罰紀錄。', source_nodes=[NodeWithScore(node=TextNode(id_='283a7310-d5fb-4efe-a9b6-aec76a2354ac', embedding=None, metadata={'sql_query': "SELECT COUNT(*) AS penalty_count FROM judgement_result WHERE CompanyName LIKE '%華盛螺絲%'", 'result': [(24,)], 'col_keys': ['penalty_count']}, excluded_embed_metadata_keys=['sql_query', 'result', 'col_keys'], excluded_llm_metadata_keys=['sql_query', 'result', 'col_keys'], relationships={}, text='[(24,)]', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'283a7310-d5fb-4efe-a9b6-aec76a2354ac': {'sql_query': "SELECT COUNT(*) AS penalty_count FROM judgement_result WHERE CompanyName LIKE '%華盛螺絲%'", 'result': [(24,)], 'col_keys': ['penalty_count']}, 'sql_query': "SELECT COUNT(*) AS penalty_count FROM judgement_result WHERE CompanyName LIKE '%華盛螺絲%'", 'result': [(24,)], 'col_keys': ['penalty_